In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
PATH='/content/drive/My Drive/corpus.txt'

In [ ]:
with open(PATH,'r') as f:
  corpus=f.read()
f.close()

In [ ]:
import re
reviews=re.split("\%\$\%|\.|\?|\!",corpus)
reviews=[review for review in reviews if review not in " "]
len(reviews)

4420131

In [ ]:
import nltk
import re
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


#fd = open('review.txt','r')
#sentences = fd.readlines()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
lemma=WordNetLemmatizer()

<h2><b>Extracting the top 20 Unigram aspects from 500000 reviews</b></h2>

In [ ]:
i=1
aspects="""{<JJ><NN.*>+},
  {<NN.*>+},
  {<VB.*><NN.*>},
  {<NN.*><VB.*>},
  {<NN><NNS>},
  
  {<NNS><VB.*>},
  {<NN.*><JJ>},
  {<NN.*><VBZ>},
  {<RB.*><JJ>+},
  {<RB><VBG>}
""".split(",")
num_dicts=len(aspects)
aspect_dicts=dict()
for j in aspects:
  aspect_dicts[j]=Counter()
for sentence in reviews[:500000]:
  sentence=sentence.lower()
  sentence=re.sub("[^a-zA-Z]+"," ",sentence)
  tokens=nltk.word_tokenize(sentence)
  
  lemmatized_tokens=[lemma.lemmatize(token) for token in tokens if token not in stop_words]
  #print(lemmatized_tokens)
  tags=nltk.pos_tag(lemmatized_tokens)
  #print(tags)
  for asp in aspects: 

    cp=nltk.RegexpParser(f"ASPECT: {asp}")
  
    res=cp.parse(tags)
    #print(res)
    for x in res:
      if type(x)==nltk.tree.Tree and x.label()=="ASPECT":
        for y in x.leaves():
          if "NN" in y[1]:
            aspect=[y[0]]
            aspect_dicts[asp].update(aspect)
    print(i,end=' sentences processed \r')
    i+=1    

In [ ]:
for j in aspects: 
  print("Rule: ",j,". Aspect: ",aspect_dicts[j].most_common(20))

Rule:  {<JJ><NN.*>+} . Aspect:  [('phone', 28337), ('case', 22813), ('product', 12387), ('time', 10498), ('battery', 9786), ('price', 8219), ('use', 7907), ('work', 7787), ('quality', 7716), ('screen', 6806), ('iphone', 6116), ('protector', 5848), ('charge', 5789), ('device', 5131), ('fit', 4988), ('charger', 4669), ('thing', 4331), ('day', 4074), ('protection', 3652), ('look', 3596)]
Rule:  
  {<NN.*>+} . Aspect:  [('phone', 90078), ('case', 56593), ('work', 27334), ('product', 22218), ('battery', 22066), ('use', 20205), ('time', 19893), ('iphone', 16146), ('screen', 14425), ('charge', 13460), ('price', 13109), ('quality', 11404), ('device', 10533), ('fit', 10482), ('charger', 9898), ('day', 9877), ('protector', 9700), ('thing', 9583), ('problem', 9454), ('look', 8810)]
Rule:  
  {<VB.*><NN.*>} . Aspect:  [('phone', 27584), ('case', 8444), ('battery', 4097), ('iphone', 3843), ('product', 3584), ('charge', 2683), ('use', 2243), ('work', 2214), ('time', 2149), ('something', 1958), ('way

<h3><b>Using Pointwise Mutual Information to find the most probable aspect phrases and classify them into Aspects, Opinions and Intensifiers</b></h3>

In [ ]:
import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize
asp_pmi=dict()
for sentence in reviews:
  text=word_tokenize(sentence.lower())
  text=[token for token in text if token not in stop_words]
  text=" ".join(text)
  text=re.sub("[^a-zA-z]+"," ",text)
  bigram_measures = nltk.collocations.BigramAssocMeasures()
  finder = BigramCollocationFinder.from_words(word_tokenize(text))

  for i in finder.score_ngrams(bigram_measures.pmi):
    if i[0] not in asp_pmi.keys():
      asp_pmi[i[0]]=+i[1]
    else:
      asp_pmi[i[0]]=i[1]
ans=[]
for i,j in asp_pmi.items():
  ans.append([i,j])
ans.sort(reverse=True,key=lambda f:f[1])

#Printing the top 50 probable aspects
for i in range(50):
  token=nltk.word_tokenize(ans[i][0][0]+" "+ans[i][0][1])
  if nltk.pos_tag(token)[0][1] in ["NN","NNS"] and nltk.pos_tag(token)[0][1] =="NN":
    print(token,"------------","Aspect")
  elif nltk.pos_tag(token)[0][1]=="RB":
    print(token,"------------","Intensifier")
  else:
    print(token,"-----------","Opinion")


['blink', 'pattern'] ------------ Aspect
['ll', 'identify'] ------------ Aspect
['mystified', 'red'] ----------- Opinion
['sequentially', 'simultaneously'] ------------ Intensifier
['proprietary', 'data'] ----------- Opinion
['speaking', 'directly'] ----------- Opinion
['game', 'frustration'] ------------ Aspect
['without', 'aggravation'] ----------- Opinion
['method', 'spell'] ------------ Aspect
['gotten', 'attached'] ----------- Opinion
['functionality', 'certain'] ------------ Aspect
['user', 'profiles'] ------------ Aspect
['number', 'blocking'] ------------ Aspect
['fact', 'currently'] ------------ Aspect
['nits', 'pick'] ----------- Opinion
['cause', 'eye'] ------------ Aspect
['tends', 'toward'] ----------- Opinion
['accurate', 'do'] ------------ Aspect
['blacklists', 'enabled'] ----------- Opinion
['bloatware', 'galling'] ------------ Aspect
['brightens', 'skin'] ----------- Opinion
['buffering', 'actually'] ----------- Opinion
['called', 'interruptions'] ----------- Opinion
[